# MoBioFP - Feature Matching

## Import Python Libraries

In [ ]:
import cv2
import math
import matplotlib.pyplot as plt

from pathlib import Path

## Define global constants

In [ ]:
SAMPLE_DIR = "../data/processed/samples/detection"

## Define Utility Functions

In [ ]:
import numpy as np

def show_images(images, titles, cmap="gray", show_axis=False, fig_size=10, sup_title=None):
    assert((titles is None) or (len(images) == len(titles)))

    num_images = len(images)
    num_cols = 4
    num_rows = math.ceil(num_images / num_cols)

    fig_height = fig_size * (num_rows / num_cols) * 1.5
    _, axes = plt.subplots(num_rows, num_cols, figsize=(fig_size, fig_height), constrained_layout=True)
    axes = axes.ravel()

    if sup_title:
        plt.suptitle(sup_title, fontsize=24)

    for idx, (image, title) in enumerate(zip(images, titles)):
        axes[idx].imshow(image, cmap=cmap)
        axes[idx].set_title(title, fontsize=12)
        axes[idx].axis("on" if show_axis else "off")
    
    # Hide the remaining subplots
    for idx in range(num_images, num_cols * num_rows):
        axes[idx].axis("off")

plt.show()

def imkpts(image, keypoints):
    result = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    return result

def orb_bf_matcher(descriptors1, descriptors2, average=False):
    # Create a brute-force matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Perform the matching between the two descriptor sets
    matches = bf.match(descriptors1, descriptors2)

    # Sort the matches based on distance
    matches = sorted(matches, key=lambda x: x.distance)

    distance = 0.0
    if average is True:
        distance = sum(m.distance for m in matches) / len(matches)
    else:
        distance = np.median([m.distance for m in matches])

    return distance

## Read Fingerprint Images

In [ ]:
images_paths = list(Path(SAMPLE_DIR).rglob("*.png"))
images = []
images_titles = []

for p in images_paths:
    img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
    images.append(img)
    images_titles.append(p.stem)

## Split Fingerprint into Probe (NO) and Gallery (WI)

In [ ]:
probe_images = []
probe_images_titles = []
gallery_images = []
gallery_images_titles = []

# Split the images into probe and gallery
for image, title in zip(images, images_titles):
    subject_id, illumination, finger_id, background, instance_id = title.split("_")

    if illumination == "o" and background == "n":
        probe_images.append(image)
        probe_images_titles.append(title)
    else:
        gallery_images.append(image)
        gallery_images_titles.append(title)

In [ ]:
show_images(probe_images, probe_images_titles, fig_size=15, sup_title="Probe (NO) Fingerprint Images")

In [ ]:
show_images(gallery_images, gallery_images_titles, fig_size=15, sup_title="Gallery (WI) Fingerprint Images")

## Extract Features

### 1. ORB (Oriented FAST and Rotated BRIEF)

REF: https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html

In [ ]:
# Initialize the ORB detector
orb = cv2.ORB_create()

# Find the keypoints and descriptors for the probe images
probe_orb_keypoints, probe_orb_descriptors = zip(*[orb.detectAndCompute(image, None) for image in probe_images])

# Find the keypoints and descriptors for the gallery images
gallery_orb_keypoints, gallery_orb_descriptors = zip(*[orb.detectAndCompute(image, None) for image in gallery_images])

# Show keypoints for each probe image
probe_orb_keypoints_images = [imkpts(image, keypoints) for image, keypoints in zip(probe_images, probe_orb_keypoints)]
show_images(probe_orb_keypoints_images, probe_images_titles, fig_size=15, sup_title="Probe (NO) ORB Keypoints")

# Show keypoints for each gallery image
gallery_orb_keypoints_images = [imkpts(image, keypoints) for image, keypoints in zip(gallery_images, gallery_orb_keypoints)]
show_images(gallery_orb_keypoints_images, gallery_images_titles, fig_size=15, sup_title="Gallery (WI) ORB Keypoints")

In [ ]:
def orb_flann_matcher(descriptors1, descriptors2):
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
    search_params = dict(checks=50)

    # Create a FLANN matcher object
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    distance = np.median([m.distance for m in good_matches])

    return distance

In [ ]:
# distances = {k : [] for k in probe_images_titles}

# for probe_desc, probe_title in zip(probe_orb_descriptors, probe_images_titles):
#     for gallery_desc, gallery_title in zip(gallery_orb_descriptors, gallery_images_titles):
#         values = {}
#         values["gallery"] = gallery_title
#         values["distances"] = orb_flann_matcher(probe_desc, gallery_desc)
#         print(f"Matching {probe_title} with {gallery_title}: Distance = {values['distances']:.2f}")
#         distances[probe_title].append(values)

# print(distances)

### 2. BRIEF (Binary Robust Independent Elementary Features) + STAR(CenSurE)

REF: https://docs.opencv.org/4.x/dc/d7d/tutorial_py_brief.html

In [ ]:
# Initiate FAST detector
star = cv2.xfeatures2d.StarDetector_create()

# Initiate BRIEF extractor
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

# Find the keypoints and descriptors for the probe images
probe_star_keypoints = [star.detect(image, None) for image in probe_images]
probe_brief_keypoints, probe_brief_descriptors = zip(*[brief.compute(image, kp) for image, kp in zip(probe_images, probe_star_keypoints)])

# Find the keypoints and descriptors for the probe images
gallery_star_keypoints = [star.detect(image, None) for image in gallery_images]
gallery_brief_keypoints, gallery_brief_descriptors = zip(*[brief.compute(image, kp) for image, kp in zip(gallery_images, gallery_star_keypoints)])

# Show keypoints for each probe image
probe_brief_keypoints_images = [imkpts(image, keypoints) for image, keypoints in zip(probe_images, probe_brief_keypoints)]
show_images(probe_brief_keypoints_images, probe_images_titles, fig_size=15, sup_title="Probe (NO) BRIEF Keypoints")

# Show keypoints for each gallery image
gallery_brief_keypoints_images = [imkpts(image, keypoints) for image, keypoints in zip(gallery_images, gallery_brief_keypoints)]
show_images(gallery_brief_keypoints_images, gallery_images_titles, fig_size=15, sup_title="Gallery (WI) BRIEF Keypoints")

### 3. SuperGlue

In [ ]:
from superpoint_superglue_deployment import Matcher

def superglue_matcher(probe_image, gallery_image):
    superglue = Matcher()

    probe_kpts, gallery_kpts, _, _, matches = superglue.match(probe_image, gallery_image)
    _, mask = cv2.findHomography(
        np.float64([probe_kpts[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2),
        np.float64([gallery_kpts[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2),
        method=cv2.USAC_MAGSAC,
        ransacReprojThreshold=5.0,
        maxIters=10000,
        confidence=0.95,
    )

    matches = np.array(matches)[np.all(mask > 0, axis=1)]
    matches = sorted(matches, key=lambda match: match.distance)
    distance = np.median([m.distance for m in matches])
    matched_image = cv2.drawMatches(
        probe_image,
        probe_kpts,
        gallery_image,
        gallery_kpts,
        matches[:12],
        None,
        flags=2,
    )

    return distance, matched_image

In [ ]:
import json

distances = {k : [] for k in probe_images_titles}

for probe_image, probe_title in zip(probe_images, probe_images_titles):
    for gallery_image, gallery_title in zip(gallery_images, gallery_images_titles):
        values = {}
        values["gallery"] = gallery_title
        values["distances"], _ = superglue_matcher(probe_image, gallery_image)
        print(f"Matching {probe_title} with {gallery_title}: Distance = {values['distances']:.2f}")
        distances[probe_title].append(values)

In [ ]:
num_probe_subjects = len(distances)

thresholds = np.arange(0.30,0.80,0.01)
DI_1_list = []
FA_list = []
GR_list = []
    
for t in thresholds:
    DI_1 = 0
    FA = 0
    GR = 0
    for probe in distances:
        probe_subject = probe.split("_")[0]
        gallery = distances[probe]
        all_dist = []
        excluding_probe_dist = []
        for g in gallery:
            all_dist.append((g['gallery'],g['distances']))
            if g['gallery'].split("_")[0] != probe_subject:
                excluding_probe_dist.append((g['gallery'],g['distances']))
        all_dist = sorted(all_dist, key = lambda x : x[1])
        excluding_probe_dist = sorted(excluding_probe_dist, key = lambda x : x[1])
        if all_dist[0][0].split("_")[0] == probe_subject and all_dist[0][1] < t:
            DI_1 += 1
        if excluding_probe_dist[0][1] < t:
            FA += 1
        else:
            GR += 1

    DI_1_list.append(DI_1)
    FA_list.append(FA)
    GR_list.append(GR)
    
DIR_1_list = np.array(DI_1_list) / num_probe_subjects
FAR_list = np.array(FA_list) / num_probe_subjects
GRR_list = np.array(GR_list) / num_probe_subjects
FRR_list = 1 - DIR_1_list
AUC = round(np.trapz(DIR_1_list,FAR_list),4)

# plt.plot(thresholds, FRR_list, label="FRR")
# plt.plot(thresholds, FAR_list, label="FAR")
# plt.plot([0,1],color="red",label="AUC = 0.5")
# plt.plot([0,0], [0,1],color="green",label="AUC = 1.0")
# plt.plot([0,1], [1,1],color="green")
# plt.plot(FAR_list, DIR_1_list, color="blue", label="AUC = " + str(AUC))


## Feature Matching

In [ ]:
# import numpy as np

# def bf_compute_distance(probe_descriptors, gallery_descriptors):
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)  # Adjusted crossCheck
#     matches = bf.knnMatch(probe_descriptors, gallery_descriptors, k=2)

#     # Lowe's ratio test
#     ratio_test = 0.7
#     good_matches = []

#     for m, n in matches:
#         if m.distance < ratio_test * n.distance:
#             good_matches.append(m.distance)

#     if good_matches:
#           # Use median for robustness
#         score = np.median(good_matches)
#     else:
#         score = np.inf  # Indicate no good matches

#     return score

# for probe_ds, gallery_ds in zip(probe_orb_descriptors, gallery_orb_descriptors):
#     print(bf_compute_distance(probe_ds, gallery_ds))

# for probe_ds, gallery_ds in zip(probe_brief_descriptors, gallery_brief_descriptors):
#     print(bf_compute_distance(probe_ds, gallery_ds))

In [ ]:
# def flann_compute_distance(probe_descriptors, gallery_descriptors):
#     index_params = dict(algorithm=1, trees=5)
#     search_params = dict(checks=50)
#     flann = cv2.FlannBasedMatcher(index_params, search_params)
#     matches = flann.knnMatch(probe_descriptors, gallery_descriptors, k=2)

#     # Lowe's ratio test
#     ratio_test = 0.75
#     good_matches = []

#     for m, n in matches:
#         if m.distance < ratio_test * n.distance:
#             good_matches.append(m.distance)

#     if good_matches:
#         score = np.median(good_matches)  # Use median for robustness
#     else:
#         score = np.inf  # Indicate no good matches

#     return score

# for probe_ds, gallery_ds in zip(probe_orb_descriptors, gallery_orb_descriptors):
#     print(flann_compute_distance(probe_ds, gallery_ds))

# for probe_ds, gallery_ds in zip(probe_brief_descriptors, gallery_brief_descriptors):
#     print(flann_compute_distance(probe_ds, gallery_ds))